# Coding Assignment
First install all the dependencies

In [34]:
!pip install requests
!pip install moviepy
!pip install sqlalchemy
!pip install pandas
!pip install python-dotenv

## Download a video
Download video file (airshow.mp4) from
`https://www.dropbox.com/sh/782f7a0hmw3mn59/AADcJpRoFB0IIfi7-nR-7Eifa?dl=0`

1. To Download a file from dropbox we need parameter dl to be 1, so we can get a zip from that public folder
2. Unzip the file to get videos
3. Save them in root folder

In [11]:
import requests

url = 'https://www.dropbox.com/sh/782f7a0hmw3mn59/AADcJpRoFB0IIfi7-nR-7Eifa?dl=1'
zip_file = 'output.zip'
r = requests.get(url, stream=True)
print(r.status_code)
with open(zip_file, "wb") as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)

200


In [12]:
import zipfile
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("./")

## Cut the video
Cut it up into 1-minute clips, save as .mov or .mp4 inside a folder named “video_clips”.

In [14]:
import os

if not os.path.exists('./report'):
    os.mkdir('./video_clips')

List all mp4 files

In [16]:
mp4_files = []
for file in os.listdir("./"):
    if file.endswith(".mp4"):
        mp4_files.append(file)

Use movie py and ffmpeg to cut videos and save them in output folder.  
Videos are named in the format `{starting_frame}thFrame` and save with extension mp4

In [22]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip

for file in mp4_files:
    load_video = VideoFileClip(file)
    fps = load_video.fps
    video_duration = load_video.duration
    total_frames = int(fps * video_duration)
    number_of_clips = int(video_duration // 60) + 1
    for i in range(number_of_clips):
        start_time = i * 60
        end_time = (i + 1) * 60
        output_path = f"./video_clips/{int(fps * start_time)}thFrame.mp4"
        ffmpeg_extract_subclip(file, start_time, end_time, targetname=output_path)


9287
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


# Create a videos Table

In [30]:
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

database=os.getenv("DATABASE")
user=os.getenv("USER")
password=os.getenv("PASSWORD")
host=os.getenv("HOST") 
port=os.getenv("PORT")
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
df = pd.DataFrame(columns=['clip_name', 'clip_file_extension', 'clip_duration', 'clip_location', 'insert_timestamp'])
for file in os.listdir("./video_clips"):
    if file.endswith(".mp4"):
        video = VideoFileClip(f"./video_clips/{file}")
        row_data = {'clip_name': file,
                        'clip_file_extension': file.split(".")[-1],
                        'clip_duration': video.duration,
                        'clip_location': f"./video_clips/{file}",
                        'insert_timestamp': pd.Timestamp.now()}
        row_df = pd.DataFrame([row_data])
        df = pd.concat([df, row_df])
df.to_sql(con=engine, name="video_data", if_exists='replace', index=False)


6

# Save as CSV

In [33]:
if not os.path.exists('./report'):
    os.mkdir('./report')
df.to_csv('./report/generated_video_files.csv', index=False)